In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb

from scipy.optimize import minimize

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [4]:
# 로컬 버전

# data_dir = Path('../input/dankook')
# feature_dir = Path('../output/feature')
# val_dir = Path('../output/oof_pred')
# test_dir = Path('../output/test_pred')
# sub_dir = Path('../output/sub')


# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [5]:
# 코렙 

from google.colab import drive
drive.mount('/content/drive')

data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

Mounted at /content/drive


In [6]:
algorithm_name = 'xgbcv'
feature_name = 'stacking1'
model_name = f'{algorithm_name}_{feature_name}_2'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_target_file = feature_dir / f'feature_target.csv'

stacking1_oof_pred_file = val_dir / f'{model_name}_oof_pred.csv'
stacking1_test_pred_file = test_dir / f'{model_name}_test_pred.csv'
stacking1_submission_file = sub_dir / f'{model_name}_submission.csv'

In [7]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

# Stacking Feature 생성

In [8]:
def load_data(model_names, oof_list, test_list, feature_names=None,number_of_versions=None):
    if number_of_versions == None or number_of_versions == 1:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2']
    elif number_of_versions == 2:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver2.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver2.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2',f'{model}_ver2_class0',f'{model}ver2_class1',f'{model}ver2_class2']

In [9]:
all_oof = []
all_test = []
feature_names = []

model_names = ['lrcv_feature','rfcv_feature','xgbcv_feature','lgbmcv_feature']
load_data(model_names, all_oof, all_test, feature_names,2)

model_names = ['lrcv_polynomial_feature','rfcv_polynomial_feature']
load_data(model_names,all_oof, all_test,feature_names,2)

model_names = ['xgbcv_polynomial_feature','lgbmcv_polynomial_feature']
load_data(model_names,all_oof, all_test,feature_names,1)

all_oof = np.column_stack(all_oof)
all_test = np.column_stack(all_test)
all_oof.shape, all_test.shape, feature_names

((319923, 42),
 (80000, 42),
 ['lrcv_feature_ver1_class0',
  'lrcv_feature_ver1_class1',
  'lrcv_feature_ver1_class2',
  'lrcv_feature_ver2_class0',
  'lrcv_featurever2_class1',
  'lrcv_featurever2_class2',
  'rfcv_feature_ver1_class0',
  'rfcv_feature_ver1_class1',
  'rfcv_feature_ver1_class2',
  'rfcv_feature_ver2_class0',
  'rfcv_featurever2_class1',
  'rfcv_featurever2_class2',
  'xgbcv_feature_ver1_class0',
  'xgbcv_feature_ver1_class1',
  'xgbcv_feature_ver1_class2',
  'xgbcv_feature_ver2_class0',
  'xgbcv_featurever2_class1',
  'xgbcv_featurever2_class2',
  'lgbmcv_feature_ver1_class0',
  'lgbmcv_feature_ver1_class1',
  'lgbmcv_feature_ver1_class2',
  'lgbmcv_feature_ver2_class0',
  'lgbmcv_featurever2_class1',
  'lgbmcv_featurever2_class2',
  'lrcv_polynomial_feature_ver1_class0',
  'lrcv_polynomial_feature_ver1_class1',
  'lrcv_polynomial_feature_ver1_class2',
  'lrcv_polynomial_feature_ver2_class0',
  'lrcv_polynomial_featurever2_class1',
  'lrcv_polynomial_featurever2_class2

In [10]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['id',target_column]).values.flatten()
y.shape

(319923,)

# Stacking

- 각 oof마다 fold별로 logloos의 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [11]:
# Xgboost
stacking_params = {
    'n_jobs' : -1, 
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
    'eta': 0.3, # learning_rate
    'booster': 'gbtree',
    'objective': 'multi:softprob',
    'num_class': 3,
    'random_state': 2020,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor'
}

In [12]:
mlogloss = []

stk_oof_pred = np.zeros((all_oof.shape[0],num_class))
stk_test_pred = np.zeros((all_test.shape[0],num_class))

kFold = StratifiedKFold(n_splits=n_splits, random_state=2020, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kFold.split(all_oof,y)):
    X_train, X_val = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    watchlist = [(dtrain,'train'), (dval, 'val')]

    xgb_clf = xgb.train(stacking_params, dtrain, 5000, evals=watchlist, early_stopping_rounds=50, verbose_eval=5000)

    dtest = xgb.DMatrix(all_test)
    stk_test_pred += xgb_clf.predict(dtest) / n_splits
    stk_oof_pred[val_idx] = xgb_clf.predict(dval)
    mlogloss.append(xgb_clf.best_score)

print('mean logloss= ', np.mean(mlogloss))

[0]	train-mlogloss:0.756572	val-mlogloss:0.757715
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[41]	train-mlogloss:0.138639	val-mlogloss:0.152598

[0]	train-mlogloss:0.756757	val-mlogloss:0.756927
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[41]	train-mlogloss:0.138933	val-mlogloss:0.151147

[0]	train-mlogloss:0.756708	val-mlogloss:0.757045
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[35]	train-mlogloss:0.140728	val-mlogloss:0.15152

[0]	train-mlogloss:0.756525	val-mlogloss:0.757024
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss h

# Ensemble

- cv score가 안정적이여서 Stacking 예측값와 함께 가중치 최적화함
- 가중치 최적화에는 scipy의 minimize 사용

In [13]:
def log_loss_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, oof_predictions):
        final_prediction += weight*prediction

    return log_loss(y, final_prediction)

In [14]:
oof_predictions = []
test_predictions = []

model_names = ['lrcv_feature','rfcv_feature','xgbcv_feature','lgbmcv_feature']
load_data(model_names, oof_predictions, test_predictions, None, 2)

model_names = ['lrcv_polynomial_feature','rfcv_polynomial_feature']
load_data(model_names, oof_predictions, test_predictions, None, 2)

model_names = ['xgbcv_polynomial_feature','lgbmcv_polynomial_feature']
load_data(model_names, oof_predictions, test_predictions, None, 1)


oof_predictions.append(stk_oof_pred)
test_predictions.append(stk_test_pred)
len(oof_predictions), oof_predictions, len(test_predictions), test_predictions

(15, [array([[0.22511735, 0.00061976, 0.02426289],
         [0.01874266, 0.0508219 , 0.18043544],
         [0.07001083, 0.0953197 , 0.08466947],
         ...,
         [0.2126908 , 0.00154683, 0.03576236],
         [0.21711876, 0.01917611, 0.01370513],
         [0.16926911, 0.05412975, 0.02660115]]),
  array([[0.20993215, 0.00090294, 0.03916491],
         [0.01504067, 0.04966523, 0.1852941 ],
         [0.05025484, 0.10054712, 0.09919804],
         ...,
         [0.18305934, 0.00218676, 0.0647539 ],
         [0.21285373, 0.02771693, 0.00942934],
         [0.15057242, 0.06960503, 0.02982255]]),
  array([[0.25  , 0.    , 0.    ],
         [0.005 , 0.11  , 0.135 ],
         [0.2475, 0.    , 0.0025],
         ...,
         [0.25  , 0.    , 0.    ],
         [0.25  , 0.    , 0.    ],
         [0.25  , 0.    , 0.    ]]),
  array([[0.25  , 0.    , 0.    ],
         [0.0175, 0.13  , 0.1025],
         [0.25  , 0.    , 0.    ],
         ...,
         [0.25  , 0.    , 0.    ],
         [0.25  , 0.

In [15]:
best_scores = []
weights = []

iteration = 100 # 여러 번 구한 가중치 값 중 Best score를 선택
for i in tqdm(range(iteration)):
    starting_values = np.random.uniform(size=len(oof_predictions))
    bounds = [(0,1)]*len(oof_predictions)
    cons = ({'type':'eq','fun':lambda w: 1-sum(w)})

    res = minimize(log_loss_func,
                   starting_values,
                   method = 'SLSQP',
                   bounds = bounds,
                   constraints = cons,
                   options={'maxiter': 500})
    best_scores.append(res['fun'])
    weights.append(res['x'])

weights = weights[np.argmin(best_scores)]
print('\n')
print('Ensemble score: {}'.format(np.min(best_scores)))
print('Best Weights: {}'.format(weights))

100%|██████████| 100/100 [1:15:54<00:00, 45.54s/it]



Ensemble score: 0.15215575098566672
Best Weights: [0.00000000e+00 7.32994224e-17 8.34942076e-02 1.69837862e-02
 1.59226413e-01 3.27824825e-01 3.15129550e-17 3.69485955e-15
 1.75862554e-18 0.00000000e+00 6.68877903e-03 1.45687803e-02
 2.25674797e-01 2.85362012e-16 1.65538411e-01]


In [16]:
final_prediction = 0

# 가중치 적용
for weight, p in zip(weights, test_predictions):
    final_prediction += weight*p

print(final_prediction)

[[1.75483973e-06 6.22846198e-02 3.11867433e-01]
 [3.74147335e-01 2.46470955e-06 4.00794537e-06]
 [6.40973010e-06 3.41540010e-02 3.39993396e-01]
 ...
 [3.74146413e-01 3.68542444e-06 3.70959552e-06]
 [4.37362283e-06 2.14285476e-02 3.52720882e-01]
 [3.74151916e-01 8.30723358e-07 1.06185092e-06]]


# 제출 파일 및 기타 파일 생성

In [17]:
# submission 파일 생성

sub = pd.read_csv(sample_file)
sub[target_column] = np.argmax(final_prediction, axis=1)
sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

In [18]:
# stcking1_oof_pred 파일 생성

np.savetxt(stacking1_oof_pred_file, stk_oof_pred, fmt='%.18f',delimiter=',')

In [19]:
# stacking1_test_pred 파일 생성

np.savetxt(stacking1_test_pred_file, stk_test_pred, fmt='%.18f', delimiter=',')

In [20]:
0.93677

0.93677